In [1]:
from pprint import pprint
from pathlib import Path
import pandas as pd
import qlib
import os
from qlib.utils import init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
from G4P import G4PModel
import gpflow
from gpflow import autoflow, params_as_tensors, ParamList
import tensorflow as tf

qlib.init(provider_uri='~/.qlib/qlib_data/cn_data')

/data2/linaiqi/anaconda3/envs/qlib/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tf_ver: 1.15.0 gpflow_ver: 1.3.0


2024-10-08 14:48:19.099920: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2024-10-08 14:48:19.136599: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299905000 Hz
2024-10-08 14:48:19.142317: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d0db8ad450 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-10-08 14:48:19.142361: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-10-08 14:48:19.146912: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-10-08 14:48:19.639438: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d0db90c940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-08 14:48:19.639486: I tensorflow/compiler/xla/s

avail devices:
/device:CPU:0
/device:XLA_CPU:0
/device:XLA_GPU:0
/device:XLA_GPU:1
/device:XLA_GPU:2
/device:XLA_GPU:3


[25527:MainThread](2024-10-08 14:48:19,673) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
INFO:qlib.Initialization:default_conf: client.
[25527:MainThread](2024-10-08 14:48:19,679) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
INFO:qlib.Initialization:qlib successfully initialized based on client settings.
[25527:MainThread](2024-10-08 14:48:19,681) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data2/linaiqi/.qlib/qlib_data/cn_data')}
INFO:qlib.Initialization:data_path={'__DEFAULT_FREQ': PosixPath('/data2/linaiqi/.qlib/qlib_data/cn_data')}


In [2]:
MARKET='csi100'
BENCHMARK = "SH000300"
EXP_NAME = 'csi100_G4P'
RETRAIN = False
CKPT_PATH = './checkpoints/'
ST_TIME = "2024-09-26"
ED_TIME = "2024-09-27"

# rid = None
# M: 300, L: 3, BS: 1024, STEP: 100
# 2.1061, rmse: 0.0225
# rid = fb5430fb3d1645d49abe7bb11ad38bd7
# M: 100, L: 1, BS: 1024, STEP: 500
# lik: 2.371366, rmse: 0.022541
# rid = '5d865f8d037f451aada437c236911ffc'
# M: 100, L: 4, BS: 128, STEP: 1000
# lik: 2.329982, rmse: 0.023591
# rid = '0846193f0add47eb98b74054bc978eee'
rid='8e54ca38d5e8478da6cf6d5c5bc8f50e'


from lilab.utils.tools import mkdir
mkdir(CKPT_PATH)

In [3]:
# df = D.features(D.instruments(MARKET), ["$close"], start_time=ST_TIME, end_time=ED_TIME)
# df.groupby("datetime").size().plot()

In [4]:
handler_kwargs = {
    "start_time": '2000-01-01',
    "end_time": ED_TIME,
    "instruments": MARKET,
    'infer_processors': [
        {
        'class': 'FilterCol',
        'kwargs': {
            'fields_group': 'feature',
            'col_list': ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5", "CORD5", "CORR10", 
                        "ROC60", "RESI10", "VSTD5", "RSQR60", "CORR60", "WVMA60", "STD5", 
                        "RSQR20", "CORD60", "CORD10", "CORR20", "KLOW", "label"
                    ]
            }
        },
        {
        'class': 'Fillna',
        }
    ],
    'learn_processors':  [
        {
        'class': 'FilterCol',
        'kwargs': {
            'fields_group': 'feature',
            'col_list': ["RESI5", "WVMA5", "RSQR5", "KLEN", "RSQR10", "CORR5", "CORD5", "CORR10", 
                        "ROC60", "RESI10", "VSTD5", "RSQR60", "CORR60", "WVMA60", "STD5", 
                        "RSQR20", "CORD60", "CORD10", "CORR20", "KLOW", 
                    ]
            }
        },
        {
        'class': 'Fillna',
        }
    ],
    'label': (["Ref($close, -2) / Ref($close, -1) - 1"], ['label'])
}

handler_conf = {
    "class": "Alpha158",
    "module_path": "qlib.contrib.data.handler",
    "kwargs": handler_kwargs,
}
hd = init_instance_by_config(handler_conf)

[25527:MainThread](2024-10-08 14:48:29,462) INFO - qlib.timer - [log.py:127] - Time cost: 9.546s | Loading data Done
INFO:qlib.timer:Time cost: 9.546s | Loading data Done
[25527:MainThread](2024-10-08 14:48:29,874) INFO - qlib.timer - [log.py:127] - Time cost: 0.020s | FilterCol Done
INFO:qlib.timer:Time cost: 0.020s | FilterCol Done
[25527:MainThread](2024-10-08 14:48:29,887) INFO - qlib.timer - [log.py:127] - Time cost: 0.011s | Fillna Done
INFO:qlib.timer:Time cost: 0.011s | Fillna Done
[25527:MainThread](2024-10-08 14:48:29,928) INFO - qlib.timer - [log.py:127] - Time cost: 0.020s | FilterCol Done
INFO:qlib.timer:Time cost: 0.020s | FilterCol Done
[25527:MainThread](2024-10-08 14:48:29,940) INFO - qlib.timer - [log.py:127] - Time cost: 0.010s | Fillna Done
INFO:qlib.timer:Time cost: 0.010s | Fillna Done
[25527:MainThread](2024-10-08 14:48:29,942) INFO - qlib.timer - [log.py:127] - Time cost: 0.478s | fit & process data Done
INFO:qlib.timer:Time cost: 0.478s | fit & process data Don

In [5]:
dataset_conf = {
    "class": "DatasetH",
    "module_path": "qlib.data.dataset",
    "kwargs": {
        "handler": hd,
        "segments": {
            "train": ("2000-01-01", "2023-12-31"),
            "test": (ST_TIME, ED_TIME),
        },
        # 'step_len': 20
    },
}
dataset = init_instance_by_config(dataset_conf)

In [6]:
# from qlib.data.dataset.handler import DataHandlerLP
# df = dataset.prepare('test', data_key=DataHandlerLP.DK_I)
# df 

In [7]:
model = init_instance_by_config(
    {
        "module_path": "G4P",
        "class": "G4PModel",
        "kwargs": {
            'M': 300, 
            'L': 2, 
            'feat_dim': 20, 
            'minibatch_size': 1024,
            'num_samples': 100,
            'iterations': 500, 
            'llh': 'GGD',
            'kernel': 'Matern32', # RBF
            'ARD': True,
            'eval_step': 1000,
            'cov_type': None
        },
    }
)

In [8]:
saver = gpflow.saver.Saver()
_ = model.build(dataset)
ctx_for_loading = gpflow.saver.SaverContext(autocompile=False)
model.DGPModel = saver.load(f'{CKPT_PATH}/{rid}.ckpt', ctx_for_loading)
model.DGPModel.clear()
for layer in model.DGPModel.layers:
    layer.needs_build_cholesky = True
model.DGPModel.layers = ParamList(model.DGPModel.layers._list)
model.DGPModel.compile()

[25527:MainThread](2024-10-08 14:48:30,401) INFO - qlib.G4PQ - [G4P.py:159] - Loading training data Done. X.shape: (437923, 20)
INFO:qlib.G4PQ:Loading training data Done. X.shape: (437923, 20)


20 20


/data2/linaiqi/anaconda3/envs/qlib/lib/python3.7/site-packages/gpflow/saver/serializers.py:47: H5pyDeprecationWarning:

The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.

/data2/linaiqi/anaconda3/envs/qlib/lib/python3.7/site-packages/gpflow/saver/serializers.py:48: H5pyDeprecationWarning:

dataset.value has been deprecated. Use dataset[()] instead.



In [9]:
with R.start(experiment_name=EXP_NAME, recorder_id=rid, resume=True):
    # Inference and saving signal
    rec = R.get_recorder()
    sr = SignalRecord(model, dataset, rec)
    sr.generate()
    pred_df = rec.load_object("pred.pkl")

[25527:MainThread](2024-10-08 14:48:32,458) INFO - qlib.workflow - [exp.py:258] - Experiment 1 starts running ...
INFO:qlib.workflow:Experiment 1 starts running ...
[25527:MainThread](2024-10-08 14:48:32,469) INFO - qlib.workflow - [recorder.py:341] - Recorder 8e54ca38d5e8478da6cf6d5c5bc8f50e starts running under Experiment 1 ...
INFO:qlib.workflow:Recorder 8e54ca38d5e8478da6cf6d5c5bc8f50e starts running under Experiment 1 ...
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
[25527:MainThread](2024-10-08 14:48:32,521) INFO - qlib.workflow - [recorder.py:374] - Fail to log the uncommitted code of $CWD(/data2/linaiqi/finance/G4PQ) when run git diff.
INFO:qlib.workflow:Fail to log the uncommitted code of $CWD(/data2/linaiqi/finance/G4PQ) when run git diff.
fatal: Not a git repository (or any parent up to mount point /data2)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
[25527:MainThread](2024-10-08 14

'The following are prediction results of the G4PModel model.'
                          score
datetime   instrument          
2024-09-26 SH600009   -0.002894
           SH600010   -0.002655
           SH600019   -0.005020
           SH600028    0.000101
           SH600030   -0.003425


[25527:MainThread](2024-10-08 14:48:34,033) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
INFO:qlib.timer:Time cost: 0.000s | waiting `async_log` Done


In [10]:
pred_df.head()

score
datetime   instrument          
2024-09-26 SH600009   -0.002894
           SH600010   -0.002655
           SH600019   -0.005020
           SH600028    0.000101
           SH600030   -0.003425

In [11]:
# strategy_config = {
#         "class": "LTopkDropoutStrategy",
#         "module_path": "lilab.qlib.strategy.signal_strategy",
#         "kwargs": {
#             "signal": pred_df,
#             "topk": 20,
#             "n_drop": 5,
#             "pred_start_time": ST_TIME,
#             "pred_end_time": ED_TIME,
#         },
# }
# strategy = init_instance_by_config(strategy_config)

In [12]:
# from qlib.backtest.position import BasePosition

# strategy.reset()
# strategy.reset_common_infra()
# strategy.reset_level_infra()
# # strategy._trade_position = BasePosition(cash=100000.0)
# strategy.generate_trade_decision()

In [13]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "lilab.qlib.strategy.signal_strategy",
        # "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "signal": '<PRED>',
            "topk": 30,
            "n_drop": 0,
            'forbid_all_trade_at_limit': False,
            "risk_degree": 1.0
            # "pred_start_time": ST_TIME,
            # "pred_end_time": ED_TIME,
        },
    },
    "backtest": {
        "start_time": ST_TIME,
        "end_time": ED_TIME,
        "account": 100000,
        "benchmark": BENCHMARK,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name=EXP_NAME, recorder_id=rid, resume=True):
    # signal-based analysis
    rec = R.get_recorder()
    sar = SigAnaRecord(rec)
    sar.generate()

    #  portfolio-based analysis: backtest
    par = PortAnaRecord(rec, port_analysis_config, "day")
    par.generate()

[25527:MainThread](2024-10-08 14:48:34,351) INFO - qlib.workflow - [exp.py:258] - Experiment 1 starts running ...
INFO:qlib.workflow:Experiment 1 starts running ...
[25527:MainThread](2024-10-08 14:48:34,365) INFO - qlib.workflow - [recorder.py:341] - Recorder 8e54ca38d5e8478da6cf6d5c5bc8f50e starts running under Experiment 1 ...
INFO:qlib.workflow:Recorder 8e54ca38d5e8478da6cf6d5c5bc8f50e starts running under Experiment 1 ...
Not a git repository
To compare two paths outside a working tree:
usage: git diff [--no-index] <path> <path>
[25527:MainThread](2024-10-08 14:48:34,428) INFO - qlib.workflow - [recorder.py:374] - Fail to log the uncommitted code of $CWD(/data2/linaiqi/finance/G4PQ) when run git diff.
INFO:qlib.workflow:Fail to log the uncommitted code of $CWD(/data2/linaiqi/finance/G4PQ) when run git diff.
fatal: Not a git repository (or any parent up to mount point /data2)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
[25527:MainThread](2024-10-08 14

{'IC': nan, 'ICIR': nan, 'Rank IC': nan, 'Rank ICIR': nan}


[25527:MainThread](2024-10-08 14:48:41,644) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[25527:MainThread](2024-10-08 14:48:41,647) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[25527:MainThread](2024-10-08 14:48:44,212) WARNING - qlib.data - [data.py:666] - load calendar error: freq=day, future=True; return current calendar!
[25527:MainThread](2024-10-08 14:48:44,215) WARNING - qlib.data - [data.py:669] - You can get future calendar by referring to the following document: https://github.com/microsoft/qlib/blob/main/scripts/data_collector/contrib/README.md
[25527:MainThread](2024-10-08 14:48:44,234) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x7f6c755c3c90>
backtest loop:   0%|          | 0/2 [00:00<?, ?it/s]/data2/linaiqi/anaconda3/envs/qlib/lib/python3.7/site-packages/qlib/utils/index_data.py:492: RuntimeWarning:

Me

'The following are analysis results of benchmark return(1day).'
                         risk
mean                 0.043470
std                  0.001694
annualized_return   10.345795
information_ratio  395.945586
max_drawdown         0.000000
'The following are analysis results of the excess return without cost(1day).'
                         risk
mean                -0.043470
std                  0.001694
annualized_return  -10.345795
information_ratio -395.945586
max_drawdown        -0.044667
'The following are analysis results of the excess return with cost(1day).'
                         risk
mean                -0.043870
std                  0.002259
annualized_return  -10.440995
information_ratio -299.543942
max_drawdown        -0.045467
'The following are analysis results of indicators(1day).'
        value
ffr  0.888889
pa   0.000000
pos  0.000000


In [14]:
# load recorder
recorder = R.get_recorder(recorder_id=rid, experiment_name=EXP_NAME)

In [15]:
# load previous results
pred_df = recorder.load_object("pred.pkl")
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

In [16]:
positions

{Timestamp('2024-09-26 00:00:00'): {'_settle_type': 'None', 'position': {'cash': 100000, 'now_account_value': 100000.0}, 'init_cash': 100000},
 Timestamp('2024-09-27 00:00:00'): {'_settle_type': 'None', 'position': {'cash': 55903.00018134098, 'now_account_value': 99920.0, 'SH600900': {'amount': 312.28429287187754, 'price': 9.283207893371582, 'weight': 0.02901321069419712, 'count_day': 1}, 'SH601899': {'amount': 681.1875174207698, 'price': 2.5705108642578125, 'weight': 0.01752401835595373, 'count_day': 1}, 'SH600028': {'amount': 663.1885784877945, 'price': 5.021196365356445, 'weight': 0.03332666212819077, 'count_day': 1}, 'SH603993': {'amount': 805.4697817093132, 'price': 3.1174352169036865, 'weight': 0.02513010271869834, 'count_day': 1}, 'SH600690': {'amount': 44.90464984245435, 'price': 67.03091430664062, 'weight': 0.030124096632898927, 'count_day': 1}, 'SZ001979': {'amount': 451.0583280515173, 'price': 4.974966526031494, 'weight': 0.022457967207206078, 'count_day': 1}, 'SH600031': {'

In [17]:
for k, v in positions.items():
    pass

In [18]:
v.position

{'cash': 55903.00018134098,
 'now_account_value': 99920.0,
 'SH600900': {'amount': 312.28429287187754,
  'price': 9.283207893371582,
  'weight': 0.02901321069419712,
  'count_day': 1},
 'SH601899': {'amount': 681.1875174207698,
  'price': 2.5705108642578125,
  'weight': 0.01752401835595373,
  'count_day': 1},
 'SH600028': {'amount': 663.1885784877945,
  'price': 5.021196365356445,
  'weight': 0.03332666212819077,
  'count_day': 1},
 'SH603993': {'amount': 805.4697817093132,
  'price': 3.1174352169036865,
  'weight': 0.02513010271869834,
  'count_day': 1},
 'SH600690': {'amount': 44.90464984245435,
  'price': 67.03091430664062,
  'weight': 0.030124096632898927,
  'count_day': 1},
 'SZ001979': {'amount': 451.0583280515173,
  'price': 4.974966526031494,
  'weight': 0.022457967207206078,
  'count_day': 1},
 'SH600031': {'amount': 49.05586009543562,
  'price': 36.325931549072266,
  'weight': 0.017834265571533826,
  'count_day': 1},
 'SH601816': {'amount': 3224.3383772429256,
  'price': 0.86

In [19]:
df = pd.DataFrame(v.position).T

In [20]:
df=df.iloc[2:]
df

,amount,price,weight,count_day
SH600900,312.284293,9.283208,0.029013,1.0
SH601899,681.187517,2.570511,0.017524,1.0
SH600028,663.188578,5.021196,0.033327,1.0
SH603993,805.469782,3.117435,0.025130,1.0
SH600690,44.904650,67.030914,0.030124,1.0
SZ001979,451.058328,4.974967,0.022458,1.0
SH600031,49.055860,36.325932,0.017834,1.0
SH601816,3224.338377,0.869946,0.028072,1.0
SH601857,8001.655444,0.314185,0.025160,1.0
SH601985,1214.835399,2.625047,0.031916,1.0


In [26]:
factor_df = D.features(D.instruments(list(df.index.values)), ["$factor", "$close / $factor"], \
                       start_time=ED_TIME, end_time=ED_TIME)
factor_df.columns = ['factor', 'raw_close']
factor_df.index = factor_df.index.get_level_values('instrument')
factor_df

,factor,raw_close
instrument,,
SH600028,0.753933,6.660000
SH600031,2.038492,17.819998
SH600690,2.226941,30.099997
SH600900,0.320221,28.990000
SH600905,0.272417,4.490000
SH601006,0.374439,6.510000
SH601398,0.719872,6.010000
SH601728,0.163666,6.330000
SH601816,0.155071,5.610000


In [27]:
df_c = pd.concat([df, factor_df], axis=1)
df_c['raw_amount'] = df_c['amount'] * df_c['factor']
df_c

,amount,price,weight,count_day,factor,raw_close,raw_amount
SH600900,312.284293,9.283208,0.029013,1.0,0.320221,28.990000,100.0
SH601899,681.187517,2.570511,0.017524,1.0,0.146802,17.509998,100.0
SH600028,663.188578,5.021196,0.033327,1.0,0.753933,6.660000,500.0
SH603993,805.469782,3.117435,0.025130,1.0,0.372453,8.370000,300.0
SH600690,44.904650,67.030914,0.030124,1.0,2.226941,30.099997,100.0
SZ001979,451.058328,4.974967,0.022458,1.0,0.443402,11.220000,200.0
SH600031,49.055860,36.325932,0.017834,1.0,2.038492,17.819998,100.0
SH601816,3224.338377,0.869946,0.028072,1.0,0.155071,5.610000,500.0
SH601857,8001.655444,0.314185,0.025160,1.0,0.037492,8.379999,300.0
SH601985,1214.835399,2.625047,0.031916,1.0,0.246947,10.630000,300.0


In [23]:
mkdir('./output')
s = k.strftime('%Y%m%d')
df.to_csv(f'./output/{s}.csv')

In [24]:
pred_df.loc['2024-09-27'].sort_values(by='score',ascending=False)

score
datetime   instrument          
2024-09-27 SH688036    0.002735
           SZ000333    0.001988
           SH688008    0.001856
           SZ300274    0.000844
           SZ300015    0.000789
           SH603993    0.000781
           SZ300782    0.000774
           SZ300896    0.000681
           SZ300014    0.000642
           SH600690    0.000532
           SZ000651    0.000492
           SH601919    0.000447
           SH600028    0.000178
           SZ300122    0.000156
           SH601899    0.000110
           SH601985    0.000109
           SH600941    0.000061
           SH601600    0.000012
           SH600900   -0.000099
           SH603259   -0.000209
           SH601398   -0.000232
           SZ300124   -0.000314
           SH601728   -0.000321
           SH688111   -0.000407
           SH603501   -0.000477
           SH688012   -0.000510
           SH601816   -0.000535
           SH601857   -0.000644
           SH600031   -0.000766
           SH601088   -0.000806
           SH600547   -0.000994
           SH601006   -0.001072
           SH601012   -0.001149
           SH600030   -0.001385
           SZ001979   -0.001497
           SZ000002   -0.001512
           SZ300760   -0.001587
           SH600050   -0.001624
           SZ000100   -0.001625
           SZ002371   -0.001628
           SZ300750   -0.001647
           SH600036   -0.001707
           SH600570   -0.001785
           SH603799   -0.001786
           SZ002179   -0.001793
           SH600436   -0.001821
           SH600010   -0.001834
           SZ002594   -0.001898
           SH600760   -0.001998
           SZ000725   -0.002085
           SH601766   -0.002151
           SH600309   -0.002179
           SZ000538   -0.002199
           SH600519   -0.002311
           SZ002129   -0.002392
           SZ002352   -0.002393
           SH600406   -0.002397
           SZ002555   -0.002405
           SH688981   -0.002422
           SZ002027   -0.002452
           SH600893   -0.002453
           SH601318   -0.002472
           SH601668   -0.002516
           SH600048   -0.002550
           SH600905   -0.002550
           SZ002049   -0.002635
           SZ000063   -0.002686
           SZ002241   -0.002712
           SH600585   -0.002814
           SZ002230   -0.002895
           SH688599   -0.002924
           SZ300308   -0.002938
           SH600111   -0.003010
           SZ000938   -0.003057
           SZ002460   -0.003063
           SH600346   -0.003075
           SZ000661   -0.003089
           SH600745   -0.003122
           SZ000301   -0.003185
           SZ002475   -0.003229
           SH601888   -0.003260
           SH603986   -0.003267
           SZ002493   -0.003305
           SZ002466   -0.003459
           SH600426   -0.003461
           SZ000792   -0.003506
           SH600009   -0.003579
           SH600089   -0.003618
           SH600176   -0.003752
           SH600019   -0.003776
           SZ002252   -0.003904
           SZ000338   -0.003940
           SH600989   -0.004077
           SZ002812   -0.004220
           SH601390   -0.004242
           SZ002415   -0.004323
           SH600438   -0.004434
           SH600276   -0.004470
           SZ002271   -0.004615
           SZ002714   -0.004717